# Esercitazione 1

Misurazione dell’overlap lessicale tra una serie di definizioni per concetti generici(quindi alti nella tassonomia)/specifici(bassi nella tassonomia) e concreti/astratti. Partendo dai dati sulle definizioni (presente nella cartella "dati" su Moodle), si richiede di calcolare la similarità 2-a-2 tra le definizioni (ad es. usando la cardinalità dell'intersezione dei lemmi normalizzata sulla lunghezza minima delle definizioni), aggregando (ed effettuando la media degli score di similarità) sulle due dimensioni (concretezza / specificità).

In [23]:
import pandas as pd
import nltk
import string
from nltk.stem import PorterStemmer
import itertools
from nltk.corpus import stopwords

## Metodi di supporto

### Normalizzazione della frase

In [24]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def normalize(sentence):
    tokens = nltk.word_tokenize(sentence)
    tokens = [token for token in tokens if token not in string.punctuation] #tolgo la punteggiatura
    tokens = [token.lower() for token in tokens] # sostituisco le maiuscole con le minuscole
    tokens = [token for token in tokens if token not in stop_words] # rimuovo le stop words 
    tokens = [stemmer.stem(token) for token in tokens] # stemmizzo
    
    return tokens

### Calcolo della similarità 2-a-2
calcolo la similarità a coppie di definizioni basandosi sull'overlap lessicale, generalmente per lo specifico e concreto si ha un overlap maggiore (circa 0,33)

In [25]:
def compare(sentence1, sentence2):
    sentence1 = set(sentence1)
    sentence2 = set(sentence2)

    return len(sentence1.intersection(sentence2)) / min(len(sentence1), len(sentence2))

In [26]:
def average_similarity(sentences):
    cols1 = []
    cols2 = []
    cols3 = []

    for sentence1, sentence2 in itertools.combinations(sentences, 2):
        cols1.append(sentence1)
        cols2.append(sentence2)
        cols3.append(compare(sentence1, sentence2))

    df = pd.DataFrame({
        "sentence 1": cols1,
        "sentence 2": cols2,
        "score": cols3
    })
    top = df['score'].quantile(q=0.95)
    bottom = df['score'].quantile(q=0.05)
    new_df = df[(df['score']> bottom) & (df['score']< top)]

    return new_df.loc[:, 'score'].mean()

## Main

In [27]:
corpus = pd.read_csv('definizioni.tsv', sep='\t', engine='python')
corpus.head(3)

,door,ladybug,pain,blurriness
0,"A construction used to divide two rooms, tempo...","small flying insect, typically red with black ...",A feeling of physical or mental distress,sight out of focus
1,"It's an opening, it can be opened or closed.","It is an insect, it has wings, red with black ...","It is a feeling, physical or emotional. It is ...","It is the absence of definite borders, shapele..."
2,"An object that divide two room, closing an hol...",An insect that can fly. It has red or orange c...,A felling that couscious beings can experince ...,A sensation felt when you can't see clearly th...


### Similarità media delle definizioni

In [28]:
# normalizzo le definizioni
corpus['door'] = corpus['door'].apply(normalize)
corpus['ladybug'] = corpus['ladybug'].apply(normalize)
corpus['pain'] = corpus['pain'].apply(normalize)
corpus['blurriness'] = corpus['blurriness'].apply(normalize)

In [29]:
for coloumn in corpus:
    print(coloumn + ": " + str(average_similarity(corpus[coloumn])))

door: 0.292891648092378
ladybug: 0.5724024060230957
pain: 0.36445809092867915
blurriness: 0.1936115826359729


### Similarità media delle definizioni aggregate (tra i due concreti ed i due astratti considerati insieme)

In [30]:
new_corpus = pd.DataFrame()
new_corpus['door_ladybug'] = pd.concat([corpus['door'], corpus['ladybug']])
new_corpus['pain_blurriness'] = pd.concat([corpus['pain'], corpus['blurriness']])
new_corpus = new_corpus.reset_index(drop=True)

In [31]:
for coloumn in new_corpus:
    print(coloumn + ": " + str(average_similarity(new_corpus[coloumn])))

door_ladybug: 0.40423663870800436
pain_blurriness: 0.22963548098109005
